# 01 – Preprocesamiento de Datos (Telco Customer Churn)
**Preparación, limpieza y enriquecimiento del dataset para análisis y modelado predictivo**

## Introducción

En el sector de telecomunicaciones, la retención de clientes es un factor crítico para la rentabilidad y sostenibilidad del negocio. El fenómeno conocido como *churn*, que se refiere a la pérdida o abandono de clientes, representa un desafío constante para las empresas.

Contar con datos de calidad y correctamente preparados es fundamental para desarrollar modelos predictivos efectivos que permitan anticipar el abandono y diseñar estrategias de retención adecuadas.

## Objetivo

El presente notebook tiene como finalidad transformar el dataset original de *Telco Customer Churn* en un conjunto de datos limpio, consistente y enriquecido. Se busca asegurar la trazabilidad y la coherencia de cada transformación aplicada, para facilitar el análisis exploratorio y el posterior desarrollo de modelos de predicción.

---

## 1. Carga y descripción inicial del dataset

En esta sección se cargan los datos originales y se realiza una primera inspección para conocer su tamaño, estructura y variables disponibles. Este paso permite anticipar posibles problemas de calidad (valores faltantes, tipos incorrectos) antes de aplicar transformaciones.


In [1]:
import pandas as pd
import numpy as np


df = pd.read_csv('../data/raw/Telco-Customer-Churn.csv')

print(f"Shape inicial: {df.shape}")
df.head()


Shape inicial: (7043, 21)


,customerID,gender,SeniorCitizen,Partner,Dependents,tenure,PhoneService,MultipleLines,InternetService,OnlineSecurity,...,DeviceProtection,TechSupport,StreamingTV,StreamingMovies,Contract,PaperlessBilling,PaymentMethod,MonthlyCharges,TotalCharges,Churn
0,7590-VHVEG,Female,0,Yes,No,1,No,No phone service,DSL,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,29.85,29.85,No
1,5575-GNVDE,Male,0,No,No,34,Yes,No,DSL,Yes,...,Yes,No,No,No,One year,No,Mailed check,56.95,1889.5,No
2,3668-QPYBK,Male,0,No,No,2,Yes,No,DSL,Yes,...,No,No,No,No,Month-to-month,Yes,Mailed check,53.85,108.15,Yes
3,7795-CFOCW,Male,0,No,No,45,No,No phone service,DSL,Yes,...,Yes,Yes,No,No,One year,No,Bank transfer (automatic),42.30,1840.75,No
4,9237-HQITU,Female,0,No,No,2,Yes,No,Fiber optic,No,...,No,No,No,No,Month-to-month,Yes,Electronic check,70.70,151.65,Yes


## 2. Diagnóstico inicial de calidad y tipos de datos

Se utiliza la función `info()` para identificar el tipo de cada variable, así como la presencia de valores faltantes. Esta inspección es clave para determinar las siguientes acciones de limpieza y transformación.


In [2]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 7043 entries, 0 to 7042
Data columns (total 21 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   customerID        7043 non-null   object 
 1   gender            7043 non-null   object 
 2   SeniorCitizen     7043 non-null   int64  
 3   Partner           7043 non-null   object 
 4   Dependents        7043 non-null   object 
 5   tenure            7043 non-null   int64  
 6   PhoneService      7043 non-null   object 
 7   MultipleLines     7043 non-null   object 
 8   InternetService   7043 non-null   object 
 9   OnlineSecurity    7043 non-null   object 
 10  OnlineBackup      7043 non-null   object 
 11  DeviceProtection  7043 non-null   object 
 12  TechSupport       7043 non-null   object 
 13  StreamingTV       7043 non-null   object 
 14  StreamingMovies   7043 non-null   object 
 15  Contract          7043 non-null   object 
 16  PaperlessBilling  7043 non-null   object 


## 3. Conversión de la variable `TotalCharges` a formato numérico

La columna `TotalCharges` representa el total facturado a cada cliente durante el periodo de su relación con la empresa. Sin embargo, esta variable fue almacenada originalmente como texto, lo cual impide realizar cálculos y análisis estadísticos adecuados.

Para corregir esto, aplicamos la función `pd.to_numeric()` con el parámetro `errors='coerce'`, que convierte los valores no numéricos a `NaN` (valores faltantes). Esta acción nos permite identificar registros con datos inválidos que requieren tratamiento específico en etapas posteriores.

In [3]:
# Forzar conversión a numérico
df['TotalCharges'] = pd.to_numeric(df['TotalCharges'], errors='coerce')

## 4. Eliminación de registros con valores nulos en `TotalCharges`

Los valores nulos identificados en la columna `TotalCharges` corresponden a clientes cuyo `tenure` es igual a cero, es decir, clientes que han contratado el servicio pero aún no han generado facturación.

Dado que estos registros no aportan información útil para el análisis y modelado, se decide eliminarlos para evitar sesgos o errores posteriores.

In [4]:
# Eliminar registros con TotalCharges nulo
df = df.dropna(subset=['TotalCharges'])


## 5. Eliminación de la variable `customerID` y registros duplicados

La columna `customerID` es un identificador único para cada cliente, pero no aporta información relevante para la predicción de *churn*. Por ello, se elimina para evitar confusión y mantener un conjunto de datos limpio.

In [5]:
df.drop(columns='customerID', inplace=True)

print(f"Registros duplicados: {df.duplicated().sum()}")
df.drop_duplicates(inplace=True)
print(f"Registros duplicados (Tras limpieza): {df.duplicated().sum()}")


Registros duplicados: 22
Registros duplicados (Tras limpieza): 0


## 6. Revisión y normalización de variables categóricas

Se identificaron las variables categóricas para revisar su cardinalidad, es decir, el número de categorías únicas que contienen. Esta revisión permite detectar posibles valores atípicos o inconsistentes que afecten el análisis y modelado.

In [6]:
cat_cols = df.select_dtypes(include='object').columns.tolist()
for col in cat_cols:
    print(f"{col}: {df[col].nunique()} unique → {df[col].unique()}")


gender: 2 unique → ['Female' 'Male']
Partner: 2 unique → ['Yes' 'No']
Dependents: 2 unique → ['No' 'Yes']
PhoneService: 2 unique → ['No' 'Yes']
MultipleLines: 3 unique → ['No phone service' 'No' 'Yes']
InternetService: 3 unique → ['DSL' 'Fiber optic' 'No']
OnlineSecurity: 3 unique → ['No' 'Yes' 'No internet service']
OnlineBackup: 3 unique → ['Yes' 'No' 'No internet service']
DeviceProtection: 3 unique → ['No' 'Yes' 'No internet service']
TechSupport: 3 unique → ['No' 'Yes' 'No internet service']
StreamingTV: 3 unique → ['No' 'Yes' 'No internet service']
StreamingMovies: 3 unique → ['No' 'Yes' 'No internet service']
Contract: 3 unique → ['Month-to-month' 'One year' 'Two year']
PaperlessBilling: 2 unique → ['Yes' 'No']
PaymentMethod: 4 unique → ['Electronic check' 'Mailed check' 'Bank transfer (automatic)'
 'Credit card (automatic)']
Churn: 2 unique → ['No' 'Yes']


In [7]:
cols_with_no_service = [
    'OnlineSecurity', 'OnlineBackup', 'DeviceProtection',
    'TechSupport', 'StreamingTV', 'StreamingMovies'
]

for col in cols_with_no_service:
    df[col] = df[col].replace({'No internet service': 'No'})

df['MultipleLines'] = df['MultipleLines'].replace({'No phone service': 'No'})


Variables como `OnlineSecurity`, `StreamingTV`, etc., contenían la categoría `"No internet service"`.  
Esa información ya está capturada en `InternetService`, por lo que se normaliza a `"No"` para reducir ruido e inconsistencias.


## 7. Creación de variables derivadas

Para enriquecer el dataset con información relevante para el análisis y modelado, se generan dos variables adicionales:

- **`tenure_group`**: segmenta la antigüedad del cliente en rangos definidos (meses), facilitando el análisis por grupos de duración de servicio.

- **`MultipleServices`**: calcula la cantidad de servicios contratados por cada cliente, funcionando como un indicador del nivel de compromiso o vinculación con la empresa.

In [8]:
# Agrupación por antigüedad
df['tenure_group'] = pd.cut(df['tenure'], bins=[0, 6, 12, 24, 48, 72],
                            labels=['0-6','6-12','12-24','24-48','48-72'])

# Conteo de servicios contratados
service_cols = ['PhoneService', 'InternetService', 'OnlineSecurity', 'OnlineBackup',
                'DeviceProtection', 'TechSupport', 'StreamingTV', 'StreamingMovies']

df['MultipleServices'] = df[service_cols].apply(lambda row: sum(val == 'Yes' for val in row), axis=1)


## 8. Verificación final y exportación del dataset limpio

Se realiza una última revisión para asegurar que no existan valores nulos ni duplicados en el dataset, garantizando la calidad y la integridad de los datos para las siguientes etapas.


In [9]:
print("Verificación final de nulos:")
print(df.isnull().sum())

print(f"Duplicados restantes: {df.duplicated().sum()}")
print(f"Dimensión final: {df.shape}")



Verificación final de nulos:
gender              0
SeniorCitizen       0
Partner             0
Dependents          0
tenure              0
PhoneService        0
MultipleLines       0
InternetService     0
OnlineSecurity      0
OnlineBackup        0
DeviceProtection    0
TechSupport         0
StreamingTV         0
StreamingMovies     0
Contract            0
PaperlessBilling    0
PaymentMethod       0
MonthlyCharges      0
TotalCharges        0
Churn               0
tenure_group        0
MultipleServices    0
dtype: int64
Duplicados restantes: 0
Dimensión final: (7010, 22)


In [10]:
df.to_csv('../data/processed/clean_telco.csv', index=False)

### Conclusión

En este notebook se ha llevado a cabo un proceso riguroso de preprocesamiento para preparar el dataset original de *Telco Customer Churn* para las etapas posteriores de análisis y modelado.  

Se realizaron las siguientes acciones clave:  
- Identificación y corrección de tipos de datos incorrectos, especialmente en variables numéricas críticas como `TotalCharges`.  
- Eliminación de registros inválidos o redundantes, asegurando la integridad y calidad del conjunto de datos.  
- Normalización de categorías para reducir ruido y simplificar la interpretación de variables categóricas.  
- Creación de variables derivadas relevantes para el análisis de clientes y su comportamiento.  
- Verificación exhaustiva de datos faltantes y duplicados, confirmando que el conjunto final está limpio y listo para análisis exploratorio.

Este trabajo garantiza una base sólida y confiable que permitirá desarrollar modelos predictivos robustos y aportar insights valiosos para la retención de clientes en el sector telecomunicaciones.

El siguiente paso consistirá en explorar en profundidad las características del dataset, identificando patrones y relaciones que orienten la selección y construcción de modelos.